In [28]:
import os
from langchain_community.embeddings import GPT4AllEmbeddings
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embeddings = GPT4AllEmbeddings()
index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

In [29]:
import os
import json
import numpy as np
import torch
from torch_geometric.data import Data
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import openai
import os
from dotenv import load_dotenv

load_dotenv()


True

In [30]:
from openai import OpenAI

In [ ]:
client = OpenAI(
    # This is the default and can be omitted
    api_key = "",
)

def ask_gpt(messages: list[dict], model="gpt-4o-mini") -> str:
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=512
    )
    return response.choices[0].message.content

# 2) System prompts (in English)
sys_res = (
    "Re-rank the list of restaurants based on the user’s preferences. "
    "Return only the restaurant names, separated by commas."
)

sys_explain = (
    "You are an expert analyst. "
    "Based on the user’s preferences and the ranking you provided, "
    "please give a brief explanation of why you produced that result."
)

sys_key = "Return only the restaurant names, separated by commas."

In [32]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

c:\Users\Nhat Minh\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [33]:
with open('edinburgh-keywords_train.json', 'r', encoding="utf-8") as f:
    train_data = json.load(f)

keywords = list(train_data['np2count'].keys())
keyword_set = set(keywords)

def extract_users(info):
    l_user, user2kw = [], []
    for ii in info:
        lus = info[ii]
        for u in lus:
            if u not in l_user:
                l_user.append(u)
                user2kw.append([])
            idx = l_user.index(u)
            user2kw[idx].append(ii)
    return l_user, user2kw


In [34]:
train_users, train_users2kw = extract_users(train_data['np2users'])

In [35]:
restaurant_set = set()
listres = []
for kw in train_data['np2rests'].keys():
    listres.extend(train_data['np2rests'][kw].keys())
restaurant_set = set(listres)

keyword_set = list(keyword_set)
restaurant_set = list(restaurant_set)
restaurants = len(listres)
num_keywords = len(keyword_set)
num_restaurants = len(restaurant_set)
a = np.zeros((num_keywords, num_restaurants))

for kw in train_data['np2rests'].keys():
    for res in train_data['np2rests'][kw].keys():
        idx_kw = keyword_set.index(kw)
        idx_res = restaurant_set.index(res)
        a[idx_kw][idx_res] = 1


In [36]:
keyword_embeddings = model.encode(list(keyword_set))

In [37]:
# Load dữ liệu test
with open('edinburgh-keywords_test.json', 'r', encoding="utf-8") as r:
    test_data = json.load(r)

user_keywords = list(test_data['np2reviews'].keys())
user_keywords_list = list(user_keywords)


In [ ]:
test_users, test_users2kw = extract_users(test_data['np2users'])
test_keywords = [kw for sublist in test_users2kw for kw in sublist]
test_keyword_embeddings = model.encode(test_keywords)
similarity_scores = cosine_similarity(test_keyword_embeddings, keyword_embeddings)


In [ ]:
filtered_keywords = []
for i, user_kw in enumerate(test_users2kw):
    updated_user_kw = []
    for kw in user_kw:
        if kw not in keyword_set:
            test_idx = test_keywords.index(kw)
            sim_scores = similarity_scores[test_idx]
            best_match_idx = np.argmax(sim_scores)
            best_match_keyword = keyword_set[best_match_idx]
            updated_user_kw.append(best_match_keyword)
        else:
            updated_user_kw.append(kw)
    filtered_keywords.append(updated_user_kw)

In [ ]:
test_users2kw = filtered_keywords

In [ ]:
results = []
for kw in test_users2kw:
    t = np.zeros((1, len(keyword_set)))
    keywords_subset = kw[:10]
    for key in keywords_subset:
        if key in keyword_set:
            idx_kw = keyword_set.index(key)
            t[0][idx_kw] = 1
    R = np.dot(t, a)
    result = np.argsort(R[0])[::-1][:10]
    results.append(result)

In [ ]:
sys_res_en = (
    "Please re-rank the list of restaurants based on the user’s preferences. "
    "Return only the restaurant names, separated by commas."
)

# System prompt for explanation
sys_explain_en = (
    "You are an expert analyst. "
    "Based on the user’s preferences and the ranking you provided, "
    "please give a brief explanation of why you produced this ranking."
)

In [ ]:
def GPT_re_rank(user_id, candidate_restaurants, user_keywords):
    prompt_rank = (
        f"The user’s preferences are: {', '.join(user_keywords[:5])}.\n"
        f"Candidate restaurants: {', '.join(candidate_restaurants[:5])}.\n"
        "Please re-rank these restaurants by how well they match the user’s preferences, "
        "and return only the restaurant names, separated by commas."
    )
    messages = [
        {"role": "system", "content": sys_res_en},
        {"role": "user",   "content": prompt_rank},
    ]
    ranked_text = ask_gpt(messages)
    re_ranked = [r.strip() for r in ranked_text.split(",") if r.strip() in candidate_restaurants]
    return re_ranked or candidate_restaurants

In [ ]:
ratings_df = pd.read_csv("edinburgh.csv")

In [ ]:
def GPT_explain_and_rerank(user_id, re_ranked_restaurants, user_keywords):
    # 1) pull the ratings for each restaurant in the current list
    rating_info = []
    for r in re_ranked_restaurants:
        match = ratings_df.loc[ratings_df["rest_id"] == r, "rating"]
        if not match.empty:
            rating_info.append(f"{r} ({match.iloc[0]:.1f})")
        else:
            rating_info.append(f"{r} (no rating)")

    # 2) build the prompt
    prompt = (
        f"The following restaurants have these average ratings: {', '.join(rating_info)}.\n"
        f"You previously ranked: {', '.join(re_ranked_restaurants[:5])}.\n"
        f"User preferences: {', '.join(user_keywords[:5])}.\n\n"
        "Please output EXACTLY in this format:\n"
        "Explanation: <brief overall rationale, citing ratings where relevant>\n"
        "Improved ranking: <comma-separated list of restaurants>\n"
        "Details:\n"
        "- <Restaurant1>: <reason, mention its rating>\n"
        "- <Restaurant2>: <reason, mention its rating>\n"
        "... (one line per restaurant in Improved ranking)"
    )

    messages = [
        {"role": "system",  "content": sys_explain_en},
        {"role": "user",    "content": prompt},
    ]
    return ask_gpt(messages)

In [ ]:
# Re-Ranking and Explanation with Side Information (Ratings)
final_results = []
final_overall_explanations = []
final_detailed_explanations = []

for idx, (user, candidate_indices) in enumerate(zip(test_users, results)):
    candidate_restaurants = [restaurant_set[i] for i in candidate_indices]
    user_kw = test_users2kw[idx]

    first_rank = GPT_re_rank(user, candidate_restaurants, user_kw)

    raw = GPT_explain_and_rerank(user, first_rank, user_kw)
    lines = [l.strip() for l in raw.splitlines() if l.strip()]

    explanation_line = next(l for l in lines if l.lower().startswith("explanation:"))
    overall_expl = explanation_line.split(":", 1)[1].strip()

    ranking_line = next(l for l in lines if l.lower().startswith("improved ranking:"))
    improved_txt = ranking_line.split(":", 1)[1].strip()
    improved_list = [r.strip() for r in improved_txt.split(",") if r.strip()]

    details = {}
    try:
        start = lines.index("Details:") + 1
    except ValueError:
        start = None

    if start is not None:
        for line in lines[start:]:
            if line.startswith("-"):
                rest, sep, reason = line[1:].partition(":")
                details[rest.strip()] = reason.strip()

    final_results.append(improved_list)
    final_overall_explanations.append(overall_expl)
    final_detailed_explanations.append(details)

    print(f"User {user} → Improved Ranking: {improved_list}")
    print(f"Overall Explanation: {overall_expl}")
    print("Details:")
    for r, expl in details.items():
        print(f"  - {r}: {expl}")
    print()

User H8mXfh5XgGCqmMLwVH7k5A → Improved Ranking: ['2tGFUtUrE0DhwlX59pbArA', 'fY1IkBnRft1KR0O2tqu7pg', 'MPzZuWpKpeLBa833zzL9IQ', 'izEBByeNB835I5makSbdew', 'SxT7tgTNxkVuk67XEl7p8g']
Overall Explanation: The ranking is primarily based on average ratings, as higher-rated restaurants are generally preferred. Since the user preferences include considerations for aesthetics and social sharing (Instagram and shop front), restaurants with better ratings likely present well visually as well as providing a satisfying experience. The restaurants are ranked from highest to lowest based on their ratings.
Details:
  - 2tGFUtUrE0DhwlX59pbArA: Highest rating of 5.0, likely the best experience overall.
  - fY1IkBnRft1KR0O2tqu7pg: Strong rating of 4.0, indicating a good balance of quality.
  - MPzZuWpKpeLBa833zzL9IQ: Also rated 4.0, providing a solid experience.
  - izEBByeNB835I5makSbdew: Mid-range rating of 3.0, suitable but not exceptional.
  - SxT7tgTNxkVuk67XEl7p8g: Lowest rating of 1.0, likely fails

In [24]:
# Hàm trả về kết quả dạng dictionary để sử dụng cho hệ thống
def generate_results(test_users, results, test_users2kw, restaurant_set, re_ranked):
    output_data = {}
    for idx, (user, restaurant_indices) in enumerate(zip(test_users, results)):
        user_data = {}
        user_keywords = test_users2kw[idx]
        candidate_restaurants = [restaurant_set[i] for i in restaurant_indices]
        re_ranked_restaurants = re_ranked[idx]
        positions = [str(i) for i in restaurant_indices]
        user_data["kw"] = user_keywords[:5]
        user_data["candidate"] = re_ranked_restaurants[:5]
        user_data["positions"] = positions[:5]
        output_data[user] = user_data
    return output_data 

result_dict = generate_results(test_users, results, test_users2kw, restaurant_set, final_results)
print(result_dict)

{'H8mXfh5XgGCqmMLwVH7k5A': {'kw': ['place', 'instagram', 'voucher', 'other friends', 'shop front'], 'candidate': ['2tGFUtUrE0DhwlX59pbArA', 'fY1IkBnRft1KR0O2tqu7pg', 'MPzZuWpKpeLBa833zzL9IQ', 'izEBByeNB835I5makSbdew', 'SxT7tgTNxkVuk67XEl7p8g'], 'positions': ['579', '730', '782', '217', '833']}, 'QZqj0zcaOXV63mVVVLAhhQ': {'kw': ['place', 'bar', 'items', 'steak', 'desserts'], 'candidate': ['n2CnBRKK82cWZ9a2OjS2xQ', 'Rwzp59f3Ia-v2V8Ku0zZWQ', 'vVqxGrqt5ALxQjJGnntpKQ', 'VEy3-SnvsKgWfGM_b2irbA', 'EJNkmXMDWurCPjOZe_WXAg'], 'positions': ['32', '382', '806', '399', '171']}, 'J-NdrqdYuaBZnD8zo9pgjg': {'kw': ['place', 'sandwiches', 'dinner', 'lot', 'batter'], 'candidate': ['Rwzp59f3Ia-v2V8Ku0zZWQ', 'ymAw53bQVppVV7T40fNgRw', 'BYwoAKKtdSeylSN7QV8oNw'], 'positions': ['343', '399', '950', '389', '746']}, 'XsIVj_AszmnnsykBrKgdHw': {'kw': ['place', 'ingredients', 'meat', 'desserts', 'people'], 'candidate': ['AQEEH7JP1DFrvFuMi6TP2Q', 'WlT8rU9YYRjGSG-gX_hnCg', 'J9rMt_V1NX49rU3YUjh_7Q', 'hSEGyy0-i1_ZXGIRy

In [25]:
# Hàm lưu kết quả vào file JSON để kiểm tra nhanh
def save_rerank_results_to_json(test_users, results, test_users2kw, restaurant_set, re_ranked, file_path='./data/Singapore_re_rank.json'):
    output_data = {}
    for idx, (user, restaurant_indices) in enumerate(zip(test_users, results)):
        user_data = {}  
        user_keywords = test_users2kw[idx]
        candidate_restaurants = [restaurant_set[i] for i in restaurant_indices]
        re_ranked_restaurants = re_ranked[idx]
        positions = [str(i) for i in restaurant_indices]
        user_data["kw"] = user_keywords[:10]
        user_data["candidate"] = re_ranked_restaurants[:10]
        user_data["positions"] = positions[:10]
        output_data[user] = user_data
    with open(file_path, mode="w", encoding="utf-8") as json_file:
        json.dump(output_data, json_file, ensure_ascii=False, indent=4)
    print(f"Results saved to: {file_path}")

save_rerank_results_to_json(test_users, results, test_users2kw, restaurant_set, final_results)


Results saved to: ./data/Singapore_re_rank.json


In [26]:
def generate_full_results(test_users, results, test_users2kw, restaurant_set, re_ranked, explanations):
    output_data = {}
    for idx, user in enumerate(test_users):
        kws = test_users2kw[idx][:5]
        cand = re_ranked[idx][:5]
        # orig = [restaurant_set[i] for i in results[idx]][:5]
        exp = explanations[idx]
        output_data[user] = {
            "kw": kws,
            "candidate": cand,
            "explanation": exp
        }
    return output_data

generate_full_results(test_users, results, test_users2kw, restaurant_set, final_results, final_detailed_explanations)

{'H8mXfh5XgGCqmMLwVH7k5A': {'kw': ['place',
   'instagram',
   'voucher',
   'other friends',
   'shop front'],
  'candidate': ['2tGFUtUrE0DhwlX59pbArA',
   'fY1IkBnRft1KR0O2tqu7pg',
   'MPzZuWpKpeLBa833zzL9IQ',
   'izEBByeNB835I5makSbdew',
   'SxT7tgTNxkVuk67XEl7p8g'],
  'explanation': {'2tGFUtUrE0DhwlX59pbArA': 'Highest rating of 5.0, likely the best experience overall.',
   'fY1IkBnRft1KR0O2tqu7pg': 'Strong rating of 4.0, indicating a good balance of quality.',
   'MPzZuWpKpeLBa833zzL9IQ': 'Also rated 4.0, providing a solid experience.',
   'izEBByeNB835I5makSbdew': 'Mid-range rating of 3.0, suitable but not exceptional.',
   'SxT7tgTNxkVuk67XEl7p8g': 'Lowest rating of 1.0, likely fails to meet expectations.'}},
 'QZqj0zcaOXV63mVVVLAhhQ': {'kw': ['place',
   'bar',
   'items',
   'steak',
   'desserts'],
  'candidate': ['n2CnBRKK82cWZ9a2OjS2xQ',
   'Rwzp59f3Ia-v2V8Ku0zZWQ',
   'vVqxGrqt5ALxQjJGnntpKQ',
   'VEy3-SnvsKgWfGM_b2irbA',
   'EJNkmXMDWurCPjOZe_WXAg'],
  'explanation': {'n2C

In [27]:
def save_full_results_to_json(test_users, results, test_users2kw, restaurant_set, re_ranked, explanations, 
                              file_path='./data/Singapore_w_sidein4.json'):
    output_data = generate_full_results(
        test_users, results, test_users2kw, restaurant_set, re_ranked, explanations
    )
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    with open(file_path, mode="w", encoding="utf-8") as f:
        json.dump(output_data, f, ensure_ascii=False, indent=4)
    print(f"Full results saved to: {file_path}")

save_full_results_to_json(test_users, results, test_users2kw, restaurant_set, final_results, final_detailed_explanations)



Full results saved to: ./data/Singapore_w_sidein4.json


In [ ]:
import re

In [ ]:
baseline_df = pd.DataFrame({
    "user": test_users,
    "restaurant_names": [
        ", ".join([restaurant_set[i] for i in idxs][:10])
        for idxs in results
    ]
})
enhanced_df = pd.DataFrame({
    "user": test_users,
    "restaurant_names": [
        ", ".join(ranks[:10])
        for ranks in final_results
    ]
})

In [ ]:
def evaluate_rerank_overlap(test_users, original_candidates, re_ranked, K=5):
    scores = {}
    for user, orig, rr in zip(test_users, original_candidates, re_ranked):
        orig_top = set(orig[:K])
        rr_top   = set(rr[:K])
        overlap = len(orig_top & rr_top) / K
        scores[user] = overlap
    return scores


In [ ]:
def evaluate_explanations(test_users, user_keywords, explanations):
    scores = {}
    for user, kws, exp in zip(test_users, user_keywords, explanations):
        tokens = re.findall(r"\w+", exp)
        length = len(tokens)
        top_kw = set(kws[:5])
        exp_tokens = set(tok.lower() for tok in tokens)
        coverage = len(top_kw & exp_tokens) / len(top_kw) if top_kw else 0.0
        scores[user] = {
            "length": length,
            "coverage": round(coverage, 3)
        }
    return scores

In [ ]:
import re 

In [ ]:
recommended_df = pd.DataFrame({
    "user": test_users,
    "restaurant_names": [", ".join(ranks[:10]) for ranks in final_results]
})


In [ ]:
#Extract groundtruth from test data
user_ground_truth = {}
np2rests = test_data['np2rests']
np2users = test_data['np2users']
for keyword, restaurants in np2rests.items():
    users = np2users.get(keyword, [])
    for user in users:
        user_ground_truth.setdefault(user, set()).update(restaurants.keys())

In [ ]:
user_ground_truth = {u: list(v) for u, v in user_ground_truth.items()}

In [ ]:
def precision_recall_at_k(actual, recommended, k):
    recommended_at_k = recommended[:k]
    hits = len(set(actual) & set(recommended_at_k))
    precision = hits / k
    recall = hits / len(actual) if actual else 0.0
    return precision, recall

def average_precision(actual, recommended, k):
    recommended_at_k = recommended[:k]
    score = 0.0
    hits = 0
    for i, r in enumerate(recommended_at_k):
        if r in actual:
            hits += 1
            score += hits / (i + 1)
    return score / min(len(actual), k) if actual else 0.0

In [ ]:
k_values = [5, 10, 20]
precision_scores = {k: [] for k in k_values}
recall_scores    = {k: [] for k in k_values}
avg_precision_scores = []

for _, row in recommended_df.iterrows():
    user = row["user"]
    recommended = row["restaurant_names"].split(", ")
    actual = user_ground_truth.get(user, [])
    
    for k in k_values:
        p, r = precision_recall_at_k(actual, recommended, k)
        precision_scores[k].append(p)
        recall_scores[k].append(r)
    
    # MAP@20
    ap20 = average_precision(actual, recommended, 20)
    avg_precision_scores.append(ap20)

print("Recommendation Quality")
for k in k_values:
    print(f"Precision@{k}: {np.mean(precision_scores[k]):.4f}")
    print(f"Recall@{k}:    {np.mean(recall_scores[k]):.4f}")
MAP = np.mean(avg_precision_scores)
print(f"Mean Average Precision (MAP@20): {MAP:.4f}")

unique_recommended = set()
for names in recommended_df["restaurant_names"]:
    unique_recommended.update(names.split(", "))

all_restaurants = set().union(*[set(d.keys()) for d in test_data['np2rests'].values()])
coverage = len(unique_recommended) / len(all_restaurants) * 100
print(f"Recommendation Coverage: {coverage:.2f}%")

Recommendation Quality
Precision@5: 0.7645
Recall@5:    0.0064
Precision@10: 0.3822
Recall@10:    0.0064
Precision@20: 0.1911
Recall@20:    0.0064
Mean Average Precision (MAP@20): 0.1695
Recommendation Coverage: 30.46%
